In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import math
from torchvision import transforms, datasets
import torchvision
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import time


In [2]:
class DarkNet(nn.Module):
    def __init__(self):

        super(DarkNet, self).__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(32),nn.LeakyReLU(0.1))
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1, bias=False),\
                                    nn.BatchNorm2d(64),nn.LeakyReLU(0.1))
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(128),nn.LeakyReLU(0.1))
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=64, kernel_size=1, stride=1, padding=0, bias=False),\
                                   nn.BatchNorm2d(64),nn.LeakyReLU(0.1))
        self.conv5 = nn.Sequential(nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(128),nn.LeakyReLU(0.1))
        self.pool5 = nn.MaxPool2d(2)
        self.conv6 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(256), nn.LeakyReLU(0.1))
        self.conv7= nn.Sequential(nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1, stride=1, padding=0, bias=False),\
                                   nn.BatchNorm2d(128), nn.LeakyReLU(0.1))
        self.conv8 = nn.Sequential(nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(256), nn.LeakyReLU(0.1))
        self.pool8 = nn.MaxPool2d(2)
        self.conv9 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),\
                                   nn.BatchNorm2d(512), nn.LeakyReLU(0.1))
        self.conv10 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False),\
                                    nn.BatchNorm2d(256), nn.LeakyReLU(0.1))
        self.conv11 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),\
                                    nn.BatchNorm2d(512), nn.LeakyReLU(0.1))
        self.conv12 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, stride=1, padding=0, bias=False),\
                                    nn.BatchNorm2d(256), nn.LeakyReLU(0.1))
        self.conv13 = nn.Sequential(nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1, bias=False),\
                                    nn.BatchNorm2d(512), nn.LeakyReLU(0.1))
        self.pool13 = nn.MaxPool2d(2)
        self.conv14 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False),\
                                    nn.BatchNorm2d(1024), nn.LeakyReLU(0.1))
        self.conv15 = nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False),\
                                    nn.BatchNorm2d(512), nn.LeakyReLU(0.1))
        self.conv16 = nn.Sequential(nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, stride=1, padding=1, bias=False),\
                                    nn.BatchNorm2d(1024), nn.LeakyReLU(0.1))
        self.conv17 = nn.Sequential(nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1, stride=1, padding=0, bias=False),\
                                    nn.BatchNorm2d(512), nn.LeakyReLU(0.1))
        self.conv18 = nn.Conv2d(in_channels=512, out_channels=30, kernel_size=3, stride=1, padding=1, bias=False)
        self.pool18 = nn.AvgPool2d(7)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.pool8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = self.conv11(x)
        x = self.conv12(x)
        x = self.conv13(x)
        x = self.pool13(x)
        x = self.conv14(x)
        x = self.conv15(x)
        x = self.conv16(x)
        x = self.conv17(x)
        x = self.conv18(x)
        x = self.pool18(x)
        x = x.view(x.size(0), -1)
        return x
    

In [3]:
batchsize = 32

train_data_transform = transforms.Compose([
        transforms.RandomResizedCrop (224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
tset_data_transform = transforms.Compose([
        transforms.RandomResizedCrop (224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

train_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/train_set',transform=train_data_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchsize, shuffle=True, num_workers=12)
 
val_dataset = torchvision.datasets.ImageFolder(root='/media/lulugay/PC/CCCV-30/test_set', transform=tset_data_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=2*batchsize, shuffle=True, num_workers=12)


In [4]:
model = DarkNet()
model.cuda()
loss_func = nn.CrossEntropyLoss()

In [5]:
def test(net,test_loader):
    corret,total = 0,0
    for inputs,labels in test_loader:
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(Variable(inputs))
        _,predicted = torch.max(outputs.data,1)
        total += labels.size(0)
        corret += (predicted == labels).sum()
    print('Accuracy of the network on the test images: %d %%' % (100 * corret / total))

def poly(base_lr,power,total_epoch,now_epoch):
    return base_lr*(1-math.pow(float(now_epoch)/float(total_epoch),power))

In [6]:
num_epochs = 100
for epoch in range(num_epochs):
    batch_size_start = time.time()
    running_loss = 0.0
    print('train epoch%d'%epoch)
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        inputs = Variable(inputs)
        lables = Variable(labels)
        lr=poly(0.01,4,num_epochs,epoch)
        optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)        #交叉熵
        loss.backward()
        optimizer.step()                          #更新权重
        running_loss += loss.data[0]
    print('test epoch%d'%epoch)
    test(model,val_loader)
    torch.save(model.state_dict(),'epoch%d_weight.pkl'%epoch)
    print('Epoch [%d/%d], Loss: %.4f,need time %.4f' % (epoch + 1, num_epochs, running_loss / (45000 / batchsize), time.time() - batch_size_start))
    

train epoch0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


test epoch0
Accuracy of the network on the test images: 26 %
Epoch [1/100], Loss: 2.7289,need time 144.8540
train epoch1
test epoch1
Accuracy of the network on the test images: 36 %
Epoch [2/100], Loss: 2.2190,need time 146.3700
train epoch2
test epoch2
Accuracy of the network on the test images: 43 %
Epoch [3/100], Loss: 1.9513,need time 146.9435
train epoch3
test epoch3
Accuracy of the network on the test images: 47 %
Epoch [4/100], Loss: 1.7792,need time 146.7968
train epoch4
test epoch4
Accuracy of the network on the test images: 50 %
Epoch [5/100], Loss: 1.6557,need time 146.8348
train epoch5
test epoch5
Accuracy of the network on the test images: 54 %
Epoch [6/100], Loss: 1.5577,need time 146.8483
train epoch6
test epoch6
Accuracy of the network on the test images: 57 %
Epoch [7/100], Loss: 1.4728,need time 146.9979
train epoch7
test epoch7
Accuracy of the network on the test images: 56 %
Epoch [8/100], Loss: 1.4152,need time 147.9532
train epoch8
test epoch8
Accuracy of the netw

test epoch67
Accuracy of the network on the test images: 75 %
Epoch [68/100], Loss: 0.6181,need time 149.3548
train epoch68
test epoch68
Accuracy of the network on the test images: 75 %
Epoch [69/100], Loss: 0.6214,need time 148.6759
train epoch69
test epoch69
Accuracy of the network on the test images: 77 %
Epoch [70/100], Loss: 0.6107,need time 149.1033
train epoch70
test epoch70
Accuracy of the network on the test images: 76 %
Epoch [71/100], Loss: 0.6025,need time 149.0437
train epoch71
test epoch71
Accuracy of the network on the test images: 77 %
Epoch [72/100], Loss: 0.6048,need time 148.3358
train epoch72
test epoch72
Accuracy of the network on the test images: 76 %
Epoch [73/100], Loss: 0.5936,need time 148.1254
train epoch73
test epoch73
Accuracy of the network on the test images: 76 %
Epoch [74/100], Loss: 0.6006,need time 148.4371
train epoch74
test epoch74
Accuracy of the network on the test images: 77 %
Epoch [75/100], Loss: 0.5886,need time 147.5925
train epoch75
test epo

In [ ]:
num_epochs = 50
for epoch in range(num_epochs):
    batch_size_start = time.time()
    running_loss = 0.0
    print('train epoch%d'%epoch)
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.cuda()
        labels = labels.cuda()
        inputs = Variable(inputs)
        lables = Variable(labels)
        lr=poly(0.0004,4,num_epochs,epoch)
        optimizer = torch.optim.SGD(model.parameters(), lr, momentum=0.9)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_func(outputs, labels)        #交叉熵
        loss.backward()
        optimizer.step()                          #更新权重
        running_loss += loss.data[0]
    print('test epoch%d'%epoch)
    test(model,val_loader)
    torch.save(model.state_dict(),'epoch%d_weight.pkl'%epoch)
    print('Epoch [%d/%d], Loss: %.4f,need time %.4f' % (epoch + 1, num_epochs, running_loss / (45000 / batchsize), time.time() - batch_size_start))
    

train epoch0


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:18: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


test epoch0
Accuracy of the network on the test images: 80 %
Epoch [1/50], Loss: 0.4000,need time 144.3617
train epoch1
test epoch1
Accuracy of the network on the test images: 80 %
Epoch [2/50], Loss: 0.4015,need time 145.4336
train epoch2
test epoch2
Accuracy of the network on the test images: 80 %
Epoch [3/50], Loss: 0.3931,need time 145.5765
train epoch3
test epoch3
Accuracy of the network on the test images: 80 %
Epoch [4/50], Loss: 0.4066,need time 145.2629
train epoch4
test epoch4
Accuracy of the network on the test images: 80 %
Epoch [5/50], Loss: 0.4004,need time 145.5823
train epoch5
test epoch5
Accuracy of the network on the test images: 81 %
Epoch [6/50], Loss: 0.4013,need time 146.3454
train epoch6
test epoch6
Accuracy of the network on the test images: 80 %
Epoch [7/50], Loss: 0.3984,need time 147.3059
train epoch7
test epoch7
Accuracy of the network on the test images: 80 %
Epoch [8/50], Loss: 0.3971,need time 146.6870
train epoch8
test epoch8
Accuracy of the network on t